# base on faceNet model

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import argparse
import facenet
import os
import sys
import math
import pickle
from sklearn.svm import SVC

In [5]:
import os
from shutil import copyfile

# show image in plot

In [6]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mimage
from matplotlib.backends.backend_pdf import PdfPages


In [11]:
mode = 'CLASSIFY'
# ".pkl"
classifier_filename = 'C:/Users/VIPLAB/models/facenet/220171229-213741.pkl'
# ".pb"
model = 'C:/Users/VIPLAB/models/facenet/20171229-213741.pb'
# dataset
# data_dir = "../xiao_face_dataset_only_face_test/"
train_data_dir = "../xiao_face_lfw_and_own/"
data_dir = "../lfw_mtcnnpy_160/"

batch_size = 1000
image_size = 160

In [12]:
classes = os.listdir(train_data_dir)
classes.sort()

In [13]:
classes

['AJ_Cook',
 'AJ_Lamas',
 'Aaron_Eckhart',
 'Aaron_Guiel',
 'Aaron_Patterson',
 'Aaron_Peirsol',
 'Aaron_Pena',
 'Aaron_Sorkin',
 'Aaron_Tippin',
 'Abba_Eban',
 'Abbas_Kiarostami',
 'Abdel_Aziz_Al-Hakim',
 'Abdel_Madi_Shabneh',
 'Abdel_Nasser_Assidi',
 'Abdoulaye_Wade',
 'Abdul_Majeed_Shobokshi',
 'Abdul_Rahman',
 'Abdulaziz_Kamilov',
 'Abdullah',
 'Abdullah_Ahmad_Badawi',
 'Abdullah_Gul',
 'Abdullah_Nasseef',
 'Abdullah_al-Attiyah',
 'Abdullatif_Sener',
 'Abel_Aguilar',
 'Abel_Pacheco',
 'Abid_Hamid_Mahmud_Al-Tikriti',
 'Abner_Martinez',
 'Abraham_Foxman',
 'Aby_Har-Even',
 'Adam_Ant',
 'Adam_Freier',
 'Adam_Herbert',
 'Adam_Kennedy',
 'Adam_Mair',
 'Adam_Rich',
 'Adam_Sandler',
 'Adam_Scott',
 'Adel_Al-Jubeir',
 'Adelina_Avila',
 'Adisai_Bodharamik',
 'Adolfo_Aguilar_Zinser',
 'Adolfo_Rodriguez_Saa',
 'Adoor_Gopalakarishnan',
 'Adrian_Annus',
 'Adrian_Fernandez',
 'Adrian_McPherson',
 'Adrian_Murrell',
 'Adrian_Nastase',
 'Adriana_Lima',
 'Adriana_Perez_Navarro',
 'Adrianna_Zuzic',
 

In [ ]:
failed_image_list = []
with tf.Graph().as_default():
      
        with tf.Session() as sess:
            
            np.random.seed(seed=666)
            
            
            dataset = facenet.get_dataset(data_dir)

            # Check that there are at least one training image per class
            for cls in dataset:
                assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')            

                 
            paths, labels = facenet.get_image_paths_and_labels(dataset)
            print('Number of classes: %d' % len(dataset))
            print('Number of images: %d' % len(paths))
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(model)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            
            for i in range(nrof_batches_per_epoch):
                print("batch %d / %d" % (i ,nrof_batches_per_epoch) )
                start_index = i*batch_size
                end_index = min((i+1)*batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, image_size)
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            classifier_filename_exp = os.path.expanduser(classifier_filename)

            if (mode=='TRAIN'):
                # Train classifier
                print('Training classifier')
                model = SVC(kernel='linear', probability=True)
                model.fit(emb_array, labels)
            
                # Create a list of class names
                class_names = [ cls.name.replace('_', ' ') for cls in dataset]

                # Saving classifier model
                with open(classifier_filename_exp, 'wb') as outfile:
                    pickle.dump((model, class_names), outfile)
                print('Saved classifier model to file "%s"' % classifier_filename_exp)
                
            elif (mode=='CLASSIFY'):
                # Classify images
                print('Testing classifier')
                with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)

                print('Loaded classifier model from file "%s"' % classifier_filename_exp)

                predictions = model.predict_proba(emb_array)
                print("model.predict_proba(emb_array) finished")

                best_class_indices = np.argmax(predictions, axis=1)
                print("best_class_indices = np.argmax(predictions, axis=1)")

                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                print("best_class_probabilities finish")
                print("start show ouput")
                for i in range(len(best_class_indices)):
                    image_name = paths[i].split("/")[-1]
                    person_name = image_name.split("\\")[0]
#                     print('ground truth name %s: my predict = %s' % (, class_names[best_class_indices[i]]))
#                     print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                    print("ori image =", image_name, " my predict =", class_names[best_class_indices[i]])

                    if(person_name != class_names[best_class_indices[i]]):
                        print(image_name)
                        failed_image_list.append(paths[i])
                        
                accuracy = np.mean(np.equal(best_class_indices, labels))
                print('Accuracy: %.3f' % accuracy)

<ipython-input-14-df904ba6e561>:13: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')


Number of classes: 5751
Number of images: 13233
Loading feature extraction model
Model filename: C:/Users/VIPLAB/models/facenet/20171229-213741.pb
Calculating features for images
batch 0 / 14
batch 1 / 14
batch 2 / 14
batch 3 / 14
batch 4 / 14
batch 5 / 14
batch 6 / 14
batch 7 / 14
batch 8 / 14
batch 9 / 14
batch 10 / 14
batch 11 / 14
batch 12 / 14
batch 13 / 14
Testing classifier
Loaded classifier model from file "C:/Users/VIPLAB/models/facenet/220171229-213741.pkl"


In [ ]:
def copy_to_fail_dir(filename, dir_name = "./failed_dir"):
    copyfile(filename, os.path.join(dir_name, filename.split("\\")[-1]))
    

In [ ]:
dir_name = "../failed_dir"
if(not os.path.exists(dir_name)):
    os.mkdir(dir_name)
for filename in failed_image_list:
    copy_to_fail_dir(filename, dir_name = dir_name)